In [0]:
%run
./_common_helpers

---
### Purpose: ###
Refine Bronze `ack_277ca_raw` into Silver `ack_277ca_events` for analytics/ML and feed latest acknowledgement timestamp into `silver.claim_headers`.

---
### Notes: ###
- Normalize identifiers, parse timestamps, drop noise columns.
- Deduplicate on `ack_id` keeping the most recent `_ingest_ts`.
- Natural keys we keep: `ack_id`, `claim_id`, `trace_number`.
- Enrichment: coalesce payer attributes from `silver.dim_payer`.

In [0]:
-- ===== 1) STAGING VIEW: typed, filtered, deduped =====
CREATE OR REPLACE TEMP VIEW stage_ack_277ca AS
WITH base AS (
  SELECT
      upper(trim(ack_id))                     AS ack_id,
      upper(trim(claim_id))                   AS claim_id,
      silver.fn_to_ts_safe(event_time)        AS ack_event_ts,
      upper(trim(functional_group_id))        AS functional_group_id,
      upper(trim(interchange_control_number)) AS interchange_control_number,
      upper(trim(payer_id))                   AS payer_id,
      /* keep friendly names; normalize empty to NULL */
      nullif(trim(payer_name), '')            AS payer_name,
      upper(trim(patient_id))                 AS patient_id,
      upper(trim(provider_tin))               AS provider_tin,
      upper(trim(status_code))                AS status_code,
      nullif(trim(status_description), '')    AS status_description,
      upper(trim(trace_number))               AS trace_number,
      '277CA'                                 AS source_system,
      _ingest_ts, ingest_id
  FROM claims360_dev.bronze.ack_277ca_raw
),
filtered AS (
  -- Exclude obviously bad rows.
  SELECT *
  FROM base
  WHERE ack_id   IS NOT NULL
    AND claim_id IS NOT NULL
    AND ack_event_ts IS NOT NULL
),
deduped AS (
  SELECT
      ack_id, claim_id, ack_event_ts, functional_group_id,
      interchange_control_number, payer_id, payer_name, patient_id,
      provider_tin, status_code, status_description, trace_number,
      source_system, _ingest_ts, ingest_id
  FROM filtered
  QUALIFY ROW_NUMBER() OVER (PARTITION BY ack_id ORDER BY _ingest_ts DESC) = 1
),
enriched AS (
  -- Optional/light: backfill payer_name/group from dim if available.
  SELECT
      d.ack_id,
      d.claim_id,
      d.ack_event_ts,
      d.functional_group_id,
      d.interchange_control_number,
      d.payer_id,
      COALESCE(d.payer_name, p.payer_name)   AS payer_name,
      p.payer_group                          AS payer_group,
      d.patient_id,
      d.provider_tin,
      d.status_code,
      d.status_description,
      d.trace_number,
      d.source_system,
      d._ingest_ts,
      d.ingest_id
  FROM deduped d
  LEFT JOIN claims360_dev.silver.dim_payer p
    ON d.payer_id = p.payer_id
)
SELECT * FROM enriched
;

In [0]:
-- ===== 2) CREATE TARGET TABLE AND MERGE: SCD1 =====
CREATE TABLE IF NOT EXISTS claims360_dev.silver.ack_277ca_events (
  ack_id                         STRING NOT NULL,
  claim_id                       STRING NOT NULL,
  ack_event_ts                   TIMESTAMP,
  functional_group_id            STRING,
  interchange_control_number     STRING,
  payer_id                       STRING,
  payer_name                     STRING,
  payer_group                    STRING,
  patient_id                     STRING,
  provider_tin                   STRING,
  status_code                    STRING,
  status_description             STRING,
  trace_number                   STRING,
  source_system                  STRING,
  _ingest_ts                     TIMESTAMP,
  ingest_id                      STRING
)
USING DELTA
CLUSTER BY (claim_id)
TBLPROPERTIES (
  delta.appendOnly = 'false'
);

MERGE INTO claims360_dev.silver.ack_277ca_events AS t
USING stage_ack_277ca AS s
ON t.ack_id = s.ack_id
WHEN MATCHED AND s._ingest_ts > t._ingest_ts THEN UPDATE SET
  t.claim_id                   = s.claim_id,
  t.ack_event_ts               = s.ack_event_ts,
  t.functional_group_id        = s.functional_group_id,
  t.interchange_control_number = s.interchange_control_number,
  t.payer_id                   = s.payer_id,
  t.payer_name                 = s.payer_name,
  t.payer_group                = s.payer_group,
  t.patient_id                 = s.patient_id,
  t.provider_tin               = s.provider_tin,
  t.status_code                = s.status_code,
  t.status_description         = s.status_description,
  t.trace_number               = s.trace_number,
  t.source_system              = s.source_system,
  t._ingest_ts                 = s._ingest_ts,
  t.ingest_id                  = s.ingest_id
WHEN NOT MATCHED THEN INSERT (
  ack_id, claim_id, ack_event_ts, functional_group_id, interchange_control_number,
  payer_id, payer_name, payer_group, patient_id, provider_tin,
  status_code, status_description, trace_number, source_system, _ingest_ts, ingest_id
) VALUES (
  s.ack_id, s.claim_id, s.ack_event_ts, s.functional_group_id, s.interchange_control_number,
  s.payer_id, s.payer_name, s.payer_group, s.patient_id, s.provider_tin,
  s.status_code, s.status_description, s.trace_number, s.source_system, s._ingest_ts, s.ingest_id
);

In [0]:
-- ===== 3) UPDATE CLAIM HEADERS: with most recent ack event ts =====

MERGE INTO silver.claim_headers t
USING (
  SELECT
      claim_id,
      MAX(ack_event_ts) AS ack_event_ts       -- latest per claim
  FROM silver.ack_277ca_events
  GROUP BY claim_id
) s
ON t.claim_id = s.claim_id
WHEN MATCHED AND (
     t.last_277ca_event_ts IS NULL
  OR s.ack_event_ts > t.last_277ca_event_ts
)
THEN UPDATE SET t.last_277ca_event_ts = s.ack_event_ts;  

In [0]:
-- ===== 4) CREATE REJECTION TABLE: using idempotency key and merge =====
CREATE TABLE IF NOT EXISTS claims360_dev.silver.ack_277ca_rejects (
  reject_key     STRING  NOT NULL,   -- idempotency key
  ack_id         STRING,
  claim_id       STRING,
  raw_event_time STRING,
  reason         STRING  NOT NULL,
  _ingest_ts     TIMESTAMP,
  _ingest_file   STRING,
  source_system  STRING
)
USING DELTA
TBLPROPERTIES (delta.appendOnly = 'true');

MERGE INTO claims360_dev.silver.ack_277ca_rejects AS t
USING (
  WITH typed AS (
    SELECT
      upper(trim(ack_id))   AS ack_id_u,
      upper(trim(claim_id)) AS claim_id_u,
      event_time            AS raw_event_time,
      silver.fn_to_ts_safe(event_time) AS _evt_ts,
      _ingest_ts, _ingest_file, _source_system
    FROM claims360_dev.bronze.ack_277ca_raw
  ),
  failed AS (
    SELECT
      ack_id_u,
      claim_id_u,
      raw_event_time,
      CASE
        WHEN ack_id_u IS NULL OR claim_id_u IS NULL THEN 'MISSING_ACK_OR_CLAIM_ID'
        WHEN _evt_ts IS NULL                       THEN 'BAD_EVENT_TIME'
        ELSE 'OTHER'
      END AS reason,
      _ingest_ts, _ingest_file, _source_system
    FROM typed
    WHERE (ack_id_u IS NULL OR claim_id_u IS NULL) OR _evt_ts IS NULL
  )
  SELECT
    /* idempotent key */
    concat_ws('|',
      coalesce(ack_id_u,'<NULL_ACK>'),
      coalesce(claim_id_u,'<NULL_CLAIM>'),
      coalesce(_ingest_file,''),
      coalesce(reason,'')
    )                   AS reject_key,
    ack_id_u            AS ack_id,
    claim_id_u          AS claim_id,
    raw_event_time,
    reason,
    _ig._ingest_ts      AS _ingest_ts,
    _ig._ingest_file    AS _ingest_file,
    _ig._source_system  AS source_system
  FROM failed _ig
) AS s
ON t.reject_key = s.reject_key
WHEN NOT MATCHED THEN INSERT (
  reject_key, ack_id, claim_id, raw_event_time, reason, _ingest_ts, _ingest_file, source_system
) VALUES (
  s.reject_key, s.ack_id, s.claim_id, s.raw_event_time, s.reason, s._ingest_ts, s._ingest_file, s.source_system
);